In [ ]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

In [ ]:
# read in known applicant files, dedupe and try to merge with applicants file
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os

from data_cleaning_functions import (trans_remov_punc, standardize_whitespace, remove_punc, remove_suffix_from_last_name,
                                     clean_names, has_award, has_suffix, get_suffix, replace_last_name, 
                                     is_year_range, str_sim, clean_med_school, clean_std_college_name, long_form_date, 
                                    correct_mispellings)

from dev import (
    APP_DATA_DIR, SUM_STAT_DIR, ATT_DATA_DIR, CARD_DATA_DIR, CORRECTIONS_DIR, AWARDS_KEYWORDS, NAME_COLS, RAW_NAME_COLS, 
    RAW_CARD_ID, RAW_INDEX_IDS, PERSON_APPLICATION_ID, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES, FEMALE_MIDDLE_NAMES, 
    PICKLE_DIR)
from merging_functions import *

OUTPUT_CSV = False 

PERSONAL_INFO = [
    'clean_first_name', 'clean_last_name', 'clean_middle_name',
    'date_of_birth', 'medical_school', 'clean_college_trans']


In [ ]:
# id column that links back to raw applicant data file
RAW_CARD_ID = 'raw_uuid'

# column where the raw id information is stored
RAW_INDEX_IDS = 'raw_card_ids'

# try to get one id per unique applicant in the dataset
PERSON_ID = 'person_uuid'
# id per deduped application-person - if someone applied multiple times, they will have multiple ids
PERSON_APPLICATION_ID = 'person_app_uuid' 
NIH_ID = 'dno'

APPLICANT_SUFFIX = '_ap'
ATTENDEE_SUFFIX = '_at'

%load_ext autoreload
%autoreload 2

%reload_ext autoreload

In [3]:
apps_filename = 'index_cards_deduped_fuzzy.csv'
# apps_filename = 'person_application_date_wide.csv'

NIH_filename = 'unique_attendees.csv'
# NIH_filename = 'NIH_attendee_deduped_raw.csv'

In [4]:
NAME_COLS = ['clean_middle_name', 'clean_last_name', 'clean_first_name']

MED_TRAINING_COLS = ['res_dates', 'intern_dates', 'residency_hospital', 'internship_hospital', 'medical_school', 'residency']


In [5]:
# import NIH raw data set
NIH_raw = pd.read_csv(os.path.join(ATT_DATA_DIR, NIH_filename)).drop_duplicates('dno')

In [6]:
# sorted(NIH_raw.columns)

In [7]:
# drop from the data set all people with eod years > 1980
# NIH = NIH_raw.loc[NIH_raw.eod_year<1980, :] 
NIH = NIH_raw
del NIH['medical_school']

In [8]:
# import cleaned, deduped applicant data set in wide form (multiple app dates as columns)
apps = pd.read_csv(os.path.join(APP_DATA_DIR, apps_filename))


In [9]:
apps.loc[:, 'res_dates'] = apps['residency_year(s)'].apply(long_form_date)

apps.loc[:, 'intern_dates'] = apps['internship_year(s)'].apply(long_form_date)

"Open"
Str. Medicine
Medicine


In [10]:
apps2 = apps.rename(
    columns={'residency_type': 'residency', 'internship_hospital_1': 'internship_hospital'})

In [11]:
NIH = NIH.rename(columns={'res_dtes':'res_dates', 'intern_dte':'intern_dates', 'intern_hos': 'internship_hospital', 
                         'res_hosp':'residency_hospital', 'clean_medical_school': 'medical_school'})
# sorted(NIH.columns)

In [12]:
NIH.loc[NIH.clean_last_name=='CHISARI', 'dno']

1013    623.0
Name: dno, dtype: float64

In [13]:
string_med_cols = ['medical_school', 'residency_hospital', 'internship_hospital', 'residency']

# apply string cleaning function to each of the string medical info columns
apps2.loc[:, string_med_cols] = apps2[string_med_cols].applymap(clean_names)

In [14]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

apps2.loc[:, 'clean_last_name'] = apps2.clean_last_name.apply(correct_name_mispellings_fnc)
apps2.loc[:, 'clean_first_name'] = apps2.clean_first_name.apply(correct_name_mispellings_fnc)

In [15]:
# there are a bunch of duplicates in apps, where application year is the same, but first name is missing
name_dups = apps2.loc[
    apps2.duplicated(
        ['clean_last_name', 'medical_school', 'application_year'], keep=False), NAME_COLS+MED_TRAINING_COLS]

In [16]:
# name_dups.sort_values(['clean_last_name'])

In [17]:
apps3 = apps2.sort_values(['clean_last_name', 'clean_first_name']).drop_duplicates(
    ['clean_last_name', 'medical_school', 'application_year'])

In [18]:
# there are also some duplicates on middle and last name, but first is missing on one of the dups, 
# so we need to drop these
dups_mask = apps3.duplicated(['clean_last_name', 'clean_middle_name', 'application_year'], keep=False)

In [19]:
apps4 = apps3.loc[((~dups_mask) | ((dups_mask) & (~pd.isnull(apps3.clean_first_name)))), :]

In [20]:
print apps4.shape
print apps3.shape

(4072, 88)
(4077, 88)


In [21]:
apps4.loc[apps4.clean_last_name=='HAAKENSTAD', NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name
59,OTTO,HAAKENSTAD,ALAN


In [22]:
apps3.loc[apps3.clean_last_name=='HAAKENSTAD', NAME_COLS+['clean_medical_school', 'address', 'city']]

,clean_middle_name,clean_last_name,clean_first_name,clean_medical_school,address,city
59,OTTO,HAAKENSTAD,ALAN,NaN,1718 7th Street South,South Fargo


In [23]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        df.loc[df[selection_type]==selection_value, t] = v

In [24]:
%%capture
change_names(
    apps4, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(
    apps4, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    apps4, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    apps4, 'clean_last_name', 'EILER', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    apps4, 'clean_last_name', 'FALCHUK', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    apps4, 'clean_last_name', 'BOYD', ['clean_first_name', 'clean_middle_name'], ['MICHAEL', 'RAY'])
change_names(
    apps4, 'clean_last_name', 'CHAPMAN', 
    ['clean_first_name', 'clean_middle_name', 'medical_school'], ['STANLEY', 'WILLETS', 'ROCHESTER'])
change_names(apps4, 'clean_last_name', 'DANFORTH', ['clean_first_name'], ['DAVID'])
change_names(apps4, 'clean_last_name', 'HUNT', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'D'])
change_names(apps4, 'clean_last_name', 'KARK', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'ADRIAN'])
change_names(apps4, 'clean_last_name', 'KEBABIAN', ['clean_first_name', 'clean_middle_name'], ['JOHN', 'WILLIS'])
change_names(apps4, 'clean_last_name', 'KNOPF', ['clean_first_name', 'clean_middle_name'], ['HARRY', 'LOUIS'])
change_names(apps4, 'clean_last_name', 'KROLIKOWSKI', ['clean_first_name', 'clean_middle_name'], ['FRANCIS', 'JOHN'])
change_names(apps4, 'clean_last_name', 'KASTL', ['clean_first_name', 'clean_middle_name'], ['DAVID', 'GENE'])
change_names(apps4, 'clean_first_name', 'JAN', ['clean_last_name'], ['KNOWLER'])
change_names(apps4, 'clean_last_name', 'KLAVEMAN', ['clean_last_name'], ['KLAEVEMAN'])
change_names(apps4, 'clean_last_name', 'MATHEW', ['clean_last_name'], ['MATTHEW'])

apps4.loc[apps4.clean_last_name=='CHESEBRO', ['clean_first_name']] = 'BRUCE'
apps4.loc[apps4.clean_last_name=='CHESEBRO', ['clean_middle_name']] = 'WILCOX'
apps4.loc[(apps4.clean_last_name=='HEALY') & (apps4.medical_school=='USC KECK'), ['clean_first_name']] = 'MARK'
apps4.loc[(apps4.clean_last_name=='HEALY') & (apps4.medical_school=='USC KECK'), ['clean_middle_name']] = 'H'

In [25]:
apps4.loc[apps4.clean_last_name=='LENN', 'clean_first_name'] = 'NICHOLAS'
apps4.loc[apps4.clean_last_name=='BRESLOW', 'clean_first_name'] = 'JAN'
apps4.loc[apps4.clean_last_name=='BRESLOW', 'clean_middle_name'] = 'LESLIE'

In [26]:
apps4.loc[(apps4.clean_last_name=='NADLER') & (pd.isnull(apps4.clean_first_name)), 'clean_first_name'] = 'LEE'
apps4.loc[(apps4.clean_last_name=='NADLER') & (apps4.clean_first_name=='LEE'), 'clean_middle_name'] = 'MARSHALL'
apps4.loc[(pd.isnull(apps4.clean_first_name)) & (apps4.clean_last_name=='ROSEN'), 'clean_first_name'] = 'HENRY'

In [27]:
apps4.loc[(apps4.clean_last_name=='NEELON'), 'clean_first_name'] = 'FRANCIS'
apps4.loc[(apps4.clean_last_name=='NEELON') , 'clean_middle_name'] = 'ALBERT'

In [28]:
apps4.loc[(apps4.clean_last_name=='NICHOLAS') , 'clean_first_name']


1906        JOHN
2637    NICHOLAS
Name: clean_first_name, dtype: object

In [29]:
NIH.loc[(NIH.clean_last_name=='ALEXANDER') & (NIH.clean_first_name=='JOHN'), 'clean_middle_name'] = 'CHARLES'

In [30]:
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_middle_name'] = 'ARTHUR'
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_first_name'] = 'EDWARD'
NIH.loc[(
        NIH.clean_last_name=='LEBOWITZ') & (
            NIH.clean_first_name=='EDWARD'), 'medical_school'] = 'ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA'

In [31]:
NIH.loc[(NIH.clean_last_name=='CHISARI'), NAME_COLS+['dno']]

,clean_middle_name,clean_last_name,clean_first_name,dno
1013,VINCENT,CHISARI,FRANCIS,623.0


In [32]:
# read in manual matches
man = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_dno_matches.xlsx'), index=False).rename(columns={'medical_school': 'dno_medical_school'})
man['medical_school'] = man.dno_medical_school.apply(clean_med_school)

In [33]:
apps4.loc[apps4.clean_last_name=='NEELON', NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name
989,ALBERT,NEELON,FRANCIS


In [34]:
t = pd.merge(left=apps4, right=man, on=['clean_last_name', 'clean_middle_name'], how='inner', suffixes=['_x', '_y'])
t.shape

(106, 94)

In [35]:
t['sim'] = t[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

t['clean_first_name'] = t['clean_first_name_y']

t_1 = t.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')

In [36]:
# t[NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]
t_1.loc[t_1.duplicated('dno', keep=False), NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]

,clean_middle_name,clean_last_name,clean_first_name,dno,medical_school_x,medical_school_y,sim


In [37]:
um = man[~man.dno.isin(t_1.dno)]

um.shape

(7, 8)

In [38]:
um

,dno,clean_first_name,clean_middle_name,clean_last_name,application_year_min,application_year_max,dno_medical_school,medical_school
9,427,JAN,LESLIE,BRESLOW,1968,1968,HARVARD MEDICAL SCHOOL,HARVARD
21,865,JULES,LEONARD,DIENSTAG,1972,1972,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,COLUMBIA
32,1413,JANE,G,GRAYSON,1975,1975,HARVARD,HARVARD
33,1486,ASHLEY,THOMSON,HAASE,1965,1965,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,COLUMBIA
51,2275,JULES,ROY,LODISH,1971,1971,CASE WESTERN,CASE WESTERN
52,2311,ANNE,NaN,LUCKY,1972,1972,YALE,YALE
67,2766,JAN,MARK,ORENSTEIN,1971,1971,STATE UNIVERSITY OF NEW YORK DOWNSTATE MEDICAL...,SUNY DOWNSTATE


In [39]:
apps2.loc[apps2.clean_last_name=='CHISARI', NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name
993,VINCENT,CHISARI,FRANCIS


In [40]:
t2 = pd.merge(left=apps4, right=um, on=['clean_last_name', 'clean_first_name'], how='inner', suffixes=['_x', '_y'])
t2.shape

# t2['sim'] = t2[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

# t2['clean_middle_name'] = t2['clean_middle_name_y']

# t2_1 = t2.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')
# t2_1.shape

(0, 94)

In [41]:
t3 = t_1[NAME_COLS+['dno', PERSON_ID]]
# t3 = pd.concat([t2_1[NAME_COLS+['dno', PERSON_ID]], t_1[NAME_COLS+['dno', PERSON_ID]]], axis=0)

In [42]:
man_dno = pd.merge(left=apps4, right=t3, on=PERSON_ID, how='left', suffixes=['_x', '_y'])
print man_dno.shape
print t3.shape
print man.shape

(4072, 92)
(96, 5)
(104, 8)


In [43]:
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno = man_dno.rename(columns={'clean_last_name_x': 'clean_last_name', 'clean_first_name_x': 'clean_first_name', 
                       'clean_middle_name_x': 'clean_middle_name'})
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno.loc[mask, 'clean_last_name'] = man_dno.loc[mask, 'clean_last_name_y']
mask = ~pd.isnull(man_dno.clean_first_name_y)
man_dno.loc[mask, 'clean_first_name'] = man_dno.loc[mask, 'clean_first_name_y']
mask = ~pd.isnull(man_dno.clean_middle_name_y)
man_dno.loc[mask, 'clean_middle_name'] = man_dno.loc[mask, 'clean_middle_name_y']

In [44]:
apps5 = man_dno.loc[pd.isnull(man_dno['dno']), :].drop(['dno', 'clean_middle_name_y', 'clean_first_name_y', 'clean_last_name_y'], axis=1) 

In [45]:
# remove females from data set
female_mask = (NIH.clean_first_name.isin(FEMALE_FIRST_NAMES)| NIH.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))  
NIH = NIH.loc[~female_mask, :]

female_mask = (apps5.clean_first_name.isin(FEMALE_FIRST_NAMES)| apps5.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))  
apps5 = apps5.loc[~female_mask, :]

In [46]:
def get_first_letter(str_var):
    if pd.isnull(str_var) or str_var=='':
        return np.nan
    return str_var[0]

In [47]:
NIH['clean_first_initial'] = NIH.clean_first_name.apply(get_first_letter)
NIH['clean_middle_initial'] = NIH.clean_middle_name.apply(get_first_letter)

In [48]:
print NIH.shape
print  apps5.shape

(3858, 35)
(3976, 88)


In [49]:
# After cleaning apps2 to match cleaning in Clean NIH Applicant notebook, we try to start merging
sims_cols = ['medical_school_sim', 'clean_middle_name_sim', 'clean_first_name_sim']

In [50]:
NIH['fuzzy_merge_col'] = NIH[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
apps5['fuzzy_merge_col'] = apps5[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
match1 = df_get_closest_matches(apps5, NIH, 'fuzzy_merge_col', suffixes=['_x', '_y']) 
print match1.shape

(3864, 122)


In [51]:
# add last name counter to each
app_counter = Counter(apps4.clean_last_name.values)
NIH_counter = Counter(NIH.clean_last_name.values)
match1['last_name_counts_x'] = match1.clean_last_name_x.apply(lambda x: app_counter[x])
match1['last_name_counts_y'] = match1.clean_last_name_y.apply(lambda x: NIH_counter[x])


In [52]:
def check_match(row):
    # address and application year match
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if row['clean_last_name_sim'] < 60 or app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 80:
        return 1
    if row['last_name_counts_x'] < 2 and row['last_name_counts_y'] < 2:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 5):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 50:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 5) and row['medical_school_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 5) and row['clean_first_name_sim'] > 80:
        return 1
    return 0


feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'clean_last_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

match2 = add_similarity_features(match1, feature_dict, check_match, suffixes=['_x', '_y'])

In [53]:
# match2.loc[(match2.is_match==2), sims_cols+['is_match', 'clean_first_name_x', 'clean_first_name_y',
#                         'clean_last_name_x', 'clean_last_name_y','medical_school_x', 'medical_school_y', PERSON_ID, NIH_ID]]

In [54]:
# select out people who match and make sure each person id and dno only 1x in data set
match3 = match2[match2.is_match==1].sort_values(['clean_last_name_x']+sims_cols, ascending=False).dropna(
    axis=0, subset=[RAW_CARD_ID])
print match3.shape

(2677, 128)


In [55]:
# match2[match2.clean_last_name_x=='ADLER']
NIH[NIH.clean_last_name=='ADLER']

,Unnamed: 0,application_year_max,application_year_min,citizenship,clean_first_initial,clean_first_name,clean_last_name,medical_school,clean_middle_initial,clean_middle_name,...,program,res_dates,residency_hospital,residency,source,ssn,supervisor,unknown,year_grad,fuzzy_merge_col
2946,0,NaN,NaN,NaN,R,RICHARD,ADLER,VANDERBILT,D,DAVID,...,CA,1970-1971,Johns Hopkins Hospital,Pediatrics,NIH EDUCATION,NaN,Whedon,NaN,1969.0,RICHARD DAVID ADLER
3058,2032,NaN,NaN,NaN,R,RICHARD,ADLER,VANDERBILT,NaN,NaN,...,CA,1960-1961,Johns Hopkins Hospital,Pediatrics,NIH EDUCATION,NaN,NaN,NaN,1959.0,RICHARD ADLER
3707,1849,NaN,NaN,NaN,S,STUART,ADLER,JOHNS HOPKINS,P,PHILLIP,...,NaN,NaN,NaN,NaN,NIH EDUCATION,NaN,NaN,NaN,1971.0,STUART PHILLIP ADLER


In [56]:
def filter_one_match_per_group(df, dedupe_col, sim_cols):
    # to merge cols should be a dict the names of the extra cols to merge in
    # values should be col names to rename
    # sim cols should be name of the columns to use as features
    # sim mask should be mask that accounts as actual mask
    # dedupe col is name of col to dedupe on

    def count_matches(id_list_arr):
        # for each id, make sure matched on 1x in data set
        # should be applied with rolling apply so takes in a dataframe and must return single value
        # unpack already matched ids from string
        current_id1 = id_list_arr[-1]
        other_matches = id_list_arr[:-1]
        is_dup = np.any(other_matches[:] == current_id1)
        if is_dup:
            return True
        return False

    # for each uuid, check for duplicates and choose best match based on sim cols
    # order of the sim cols should be with most important first
    dup_flag = '{}_duplicate'.format(dedupe_col)
    df[dup_flag] = 0
    df.loc[:, dup_flag] = df[
        dedupe_col].expanding(center=False, min_periods=0).apply(func=count_matches)

    df_matches = df[df['is_match'] == 1].sort_values([dedupe_col] + sim_cols, ascending=False)
    return df_matches.drop_duplicates([dedupe_col], keep='first')


In [57]:
match4 = filter_one_match_per_group(match3, 'raw_uuid', sims_cols)
print match4.shape

(2623, 129)


In [58]:
match5 = filter_one_match_per_group(match4, NIH_ID, sims_cols)
print match5.shape

(2619, 130)


In [59]:
# get nonmatched NIH people and not matched applicants 
nm_apps = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match5[RAW_CARD_ID].dropna().values)
nm_NIH = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match5[NIH_ID].dropna().values)

(1357, 88)
(1239, 36)


In [60]:
# do another round of matching just on last name
nm_match1 = df_get_closest_matches(nm_apps, nm_NIH, 'clean_last_name', suffixes=['_x', '_y']) 
print nm_match1.shape


(2031, 122)


In [61]:
def check_match(row):
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    max_name_sim = max(row['clean_first_name_sim'], row['medical_school_sim'], row['clean_middle_name_sim'])
    if max_name_sim < 60:
        return 0
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 60:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 6):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 40:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 6) and row['medical_school_sim'] > 80:
        return 1
    if not pd.isnull(row['medical_school_sim']) and row['medical_school_sim'] < 40:
        return 0
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_first_name_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_middle_name_sim'] > 80:
        return 1
    return 0

In [62]:
nm_feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

nm_match2 = add_similarity_features(nm_match1, nm_feature_dict, check_match, suffixes=['_x', '_y'])


In [63]:
nm_match3 = nm_match2.loc[(nm_match2.is_match==1) & (nm_match2.index!='MORTON'), :].reset_index(
    drop=False).rename(columns={'index': 'clean_last_name'})

In [64]:
# merge the manual people with dno with the dno data set and append to the matches list
man_dno2 = man_dno.loc[~pd.isnull(man_dno['dno']), :].drop(['clean_first_name_y', 'clean_middle_name_y',
                                                            'clean_last_name_y'], axis=1)
man_dno2.loc[:, 'dno'] = man_dno2.dno.astype(int)
NIH.loc[:, 'dno'] = NIH.dno.astype(int)

In [65]:
print man_dno2.shape
man_dno_merge = pd.merge(left=man_dno2, right=NIH, on='dno', how='inner')
print man_dno_merge.shape

(96, 89)
(95, 124)


In [66]:
man_dno_merge2 = man_dno_merge.rename(columns={'clean_first_name_x': 'clean_first_name2', 
                                               'clean_middle_name_x': 'clean_middle_name2',
                             'clean_last_name_x': 'clean_last_name2'})


In [67]:
fails = man_dno2.loc[~man_dno2.dno.isin(man_dno_merge.dno), :]
print fails.shape

(1, 89)


In [68]:
# append matches together
match6 = pd.concat([nm_match3, match5, man_dno_merge2], axis=0)

In [69]:
# get nonmatched NIH people and not matched applicants 
nm_apps2 = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match6[RAW_CARD_ID].dropna().values)
nm_NIH2 = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match6[NIH_ID].dropna().values)

(1313, 88)
(1117, 36)


In [70]:
nm_NIH3 = nm_NIH2.loc[
    (nm_NIH2.eod_year< 1976) & (
        nm_NIH2.eod_year>1963), NAME_COLS+['medical_school', 'eod_year']].sort_values('clean_last_name')
print nm_NIH3.shape

(59, 5)


In [71]:
apps_match = nm_apps2.loc[nm_apps2.clean_last_name.isin(nm_NIH3.clean_last_name.values)]
test_merge = pd.merge(left=nm_NIH3, right=apps_match, on='clean_last_name', how='inner').sort_values('clean_last_name')
test_merge = test_merge[sorted(test_merge.columns)]
if OUTPUT_CSV:
    test_merge.to_csv(os.path.join(CORRECTIONS_DIR, 'test_merge_missing_NIH.csv'), index=False)

In [72]:
# test_merge[['clean_first_name_x', 'clean_first_name_y', 'clean_middle_name_x', 'clean_middle_name_y']]

In [73]:
match6.loc[pd.isnull(match6.clean_last_name), 'clean_last_name'] =  match6.loc[
    pd.isnull(match6.clean_last_name), 'clean_last_name_x']
to_drop = [c for c in match6.columns if c.endswith('_sim') or '_counts' in c]
match7= match6.drop(to_drop+[
        'dup_flag', 'eod_year_diff', 'fuzzy_merge_col_x', 'fuzzy_merge_col_y', 'unknown',
        'Unnamed: 0', 'raw_uuid_duplicate', 'dno_duplicate', 'count_missing',
                'clean_last_name_x', 'clean_last_name_y', 'is_match'], axis=1)

In [74]:
# consolidate columns in match6
match7a = consolidate_merge_cols(match7, ['_x', '_y'], [])

['citizenship', 'clean_first_initial', 'clean_first_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'intern_dates', 'internship_hospital', 'medical_school', 'res_dates', 'residency_hospital', 'residency']


In [75]:
mask = ~pd.isnull(match7a.clean_first_name2)
match7a.loc[mask, 'clean_first_name'] = match7a.loc[mask, 'clean_first_name2']

mask = ~pd.isnull(match7a.clean_last_name2)
match7a.loc[mask, 'clean_last_name'] = match7a.loc[mask, 'clean_last_name2']

mask = ~pd.isnull(match7a.clean_middle_name2)
match7a.loc[mask, 'clean_middle_name'] = match7a.loc[mask, 'clean_middle_name2']

In [76]:
match8 = pd.concat([
        man_dno[~pd.isnull(man_dno['dno'])], match7a, apps4.loc[
            ~apps4[PERSON_ID].isin(match7a[PERSON_ID].values),:]], axis=0).drop(['clean_first_name2', 
                                                                                'clean_middle_name2', 
                                                                                'clean_last_name2'], axis=1)
print sorted(match8.columns)
print match8.shape

['address', 'age', 'application_date', 'application_year', 'application_year_1', 'application_year_2', 'application_year_3', 'application_year_max', 'application_year_min', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_initial', 'clean_first_name', 'clean_first_name_y', 'clean_last_name', 'clean_last_name_y', 'clean_middle_initial', 'clean_middle_name', 'clean_middle_name_y', 'clean_suffix', 'clinical', 'cord', 'data_source', 'date_of_birth', 'dbs', 'dno', 'dno_medical_school', 'dob', 'duplicate_dno', 'eod_year', 'fifth', 'first_name', 'fuzzy_merge_col', 'generation', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'index', 'institute', 'intern_dates', 'internship_hospital', 'internship_year(s)', 'is_female', 'is_match', 'lab_brch', 'last_name', 'last_name_counts', 'level_0', 'med_school', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', '

In [77]:
# replace date of birth with dob whenever date of birth missing and dob is not
match8.loc[
    (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'date_of_birth'] = match8.loc[
        (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'dob']


c1 = 'date_of_birth'
c2 = 'dob'
match8.loc[(pd.isnull(match8[c1])) & (~pd.isnull(match8[c2])), [c1, c2]]

,date_of_birth,dob


In [78]:
match8.loc[(match8.clean_last_name=='KNOWLER') & (match8.clean_first_name=='JAN'), 'dno'] = 1922
match8.loc[
    (match8.clean_last_name== 'E ROSS HARVARD') & (match8.clean_first_name=='MICHAEL'), 'clean_last_name'] = 'ROSS'


In [131]:
match9 = match8.drop(['dob'], axis=1).reset_index(drop=True).sort_values(
    NAME_COLS+['dno']).drop_duplicates(NAME_COLS+['dno']).drop_duplicates(NAME_COLS+[PERSON_ID])
# print sorted(match9.columns)
print match8.shape
print match9.shape

(4170, 109)
(4070, 108)


In [133]:
match9.original_medical_school.dropna().unique()

array(['University of Iowa College of Medicine',
       'Columbia University College of Physicians and Surgeons',
       'University of Washington School of Medicine', 'other',
       'Joan & Sanford I. Weill Medical College Cornell University',
       'University of Texas Southwestern Medical Center at Dallas',
       'University of Pennsylvania School of Medicine',
       'Georgetown University School of Medicine',
       'Duke University School of Medicine',
       'University of Chicago Pritzker School of Medicine',
       'SUNY Buffalo School of Medicine & Biomedical Sciences',
       'Harvard Medical School',
       'Loma Linda University School of Medicine',
       'Baylor College of Medicine',
       'University of Miami School of Medicine',
       'University of Rochester School of Medicine and Dentistry',
       'Northwestern University Medical School',
       'New York University School of Medicine',
       'Wake Forest University School of Medicine',
       'SUNY Downstate 

In [141]:
match9.loc[match9.medical_school=='WASHINGTON', NAME_COLS+['dno_medical_school', 'original_medical_school', 'medical_school', 'clean_medical_school']]

,clean_middle_name,clean_last_name,clean_first_name,dno_medical_school,original_medical_school,medical_school,clean_medical_school
758,RAPHAEL,ROSENTHAL,RICHARD,WASHINGTON,SUNY Downstate Medical Center College of Medicine,WASHINGTON,OTHER


In [136]:
# create a clean (standardized medical school column and remove people with international med schools)
match9['medical_school'] = match9['dno_medical_school']
mask = pd.isnull(match9.medical_school)
match9.loc[mask, 'medical_school'] = match9.loc[mask, 'original_medical_school']


match9['clean_medical_school'] = match9.medical_school.apply(clean_med_school)

OTHER
OTHER
OTHER
OTHER
TUFTS
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
PENNSLYVANIA
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
WASHINGTON
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER
OTHER


In [130]:
match9.loc[match9.original_medical_school.apply(lambda x: False if pd.isnull(x) else 'Toledo' in x), :]



,address,age,application_date,application_year,application_year_1,application_year_2,application_year_3,application_year_max,application_year_min,associate_program_entered,...,supervisor,teaching,undergrad_year_grad,undergraduate_school,withdrawal,year_accepted,year_grad,zip_code,control_flag,clean_medical_school


In [124]:
match9.loc[match9.original_medical_school == '', ['medical_school', 'dno_medical_school', 'original_medical_school']]

,medical_school,dno_medical_school,original_medical_school


In [ ]:
# drop people who went to international medical schools
from dev import INTERNATIONAL_MED_SCHOOLS

def is_international(med_str, lst_itl_schools):
    

In [80]:
match9['control_flag'] = 0
match9.loc[pd.isnull(match9.dno) & pd.isnull(match9.year_accepted), 'control_flag'] = 1

In [81]:
match9.rename(columns={'res_dates': 'residency_dates', 'intern_dates': 'internship_dates', 
                      'clean_college_trans': 'clean_college'}, inplace=True)

IMPORTANT_COLS = [NIH_ID, PERSON_ID, 'application_year', 'eod_year', 'application_date', 'clean_first_name', 'clean_middle_name', 
                 'clean_last_name', 'control_flag', 'year_accepted', 'rejected', 'rejection_date', 'clean_college', 'medical_school',
                'residency_dates', 'internship_dates']

other_cols = sorted([i for i in match9.columns if i not in IMPORTANT_COLS])

# order columns so important ones are 
match10 = match9[IMPORTANT_COLS+other_cols].sort_values(['clean_last_name', 'application_year'])

match11 = match10.dropna(subset=[PERSON_ID], axis=0).sort_values(['clean_last_name', 'clean_first_name'])

# wide_apps5.to_pickle(os.path.join(APP_DATA_DIR, 'all_apps_plus_NIH_info.p'))
match11.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info.csv'), index=False)

wide_apps_v = match11.loc[(match11.application_year>1960) & (match11.application_year<1976), :].sort_values(
    ['clean_last_name', 'application_date'])

wide_apps_v.to_pickle(os.path.join(PICKLE_DIR, 'fuzzy_all_apps_plus_NIH_info_vietnam.p'))

wide_apps_v.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info_vietnam.csv'), index=False)

In [86]:
ln = 'RING'
wide_apps_v.loc[wide_apps_v.clean_last_name==ln, NAME_COLS+[PERSON_ID, 'medical_school', 'application_year', 'dno', 'residency','residency_hospital',
                                                            'internship_hospital', 'residency_dates']]

,clean_middle_name,clean_last_name,clean_first_name,person_uuid,medical_school,application_year,dno,residency,residency_hospital,internship_hospital,residency_dates
3847,STEVES,RING,W,3807.0,HARVARD,1971.0,NaN,SURGERY,DUKE UNIVERSITY,DUKE UNIVERSITY,1972-1973


In [ ]:
wide_apps_v.loc[wide_apps_v.clean_last_name=='KRUGMAN', NAME_COLS+['dno', PERSON_ID]]

In [ ]:
wide_apps_v.loc[pd.isnull(wide_apps_v.clean_first_name), NAME_COLS+['dno', 'medical_school', 'application_year']].to_csv('extras.csv')

In [ ]:
apps5.loc[apps5.clean_last_name=='PRZYBYLA', NAME_COLS+['medical_school', PERSON_ID]]